### A python script to calculate and plot Temperature Advection by Samuel Ekwacu

In [323]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.units import units
import numpy as np
import xarray as xr
from metpy.cbook import example_data

In [324]:
#Read mean temperature data
ds =xr.open_dataset('C:/Diabatic_data/extra2/advection/temp1_1961-1981_mon.nc')
ds = ds.sel(height=2) #selecting to the pressure level
#ds.data_vars

In [325]:
#seting coordinate system for temp
ds.coords['lon']=(ds.coords['lon']+180)%360-180
ds=ds.sortby(ds.lon)
#ds.data_vars
#ds

In [326]:
temp= ds.sel(lon=slice(-20, 55), lat=slice(40, -35))  ## Slice to Africa
temp.data_vars
#temp

Data variables:
    air_temp  (time, lat, lon) float32 ...

In [327]:
#Read mean uv components of wind
ds1 =xr.open_dataset('C:/Diabatic_data/extra2/advection/mean_uvwind1_1961-1981.nc')
ds1 = ds1.sel(lev=85000) #selecting to the pressure level
#ds1.data_vars

In [328]:
#seting coordinate system for uv componnets of wind
ds1.coords['lon']=(ds1.coords['lon']+180)%360-180
ds1=ds1.sortby(ds1.lon)
#ds1.data_vars

In [329]:
uv_wind = ds1.sel(lon=slice(-20, 55), lat=slice(40, -35))  ## Slice to Africa
uv_wind.data_vars
#uv_wind

Data variables:
    u        (time, lat, lon) float32 ...
    v        (time, lat, lon) float32 ...

In [330]:
# Get lat/lon data from file
temp_var = temp.variables['air_temp']
u_wind_var = uv_wind.variables['u']
v_wind_var = uv_wind.variables['v']
#time_var = temp.variables[find_time_var(temp_var)]
lat_var = temp.variables['lat']
lon_var = temp.variables['lon']

In [331]:
# Get actual data values and remove any size 1 dimensions
lat = lat_var[:].squeeze()
lon = lon_var[:].squeeze()
#hght = hght_var[:].squeeze()
temp1 = temp_var[:].squeeze() * units.degC
u_wind = units('m/s') * u_wind_var[:].squeeze()
v_wind = units('m/s') * v_wind_var[:].squeeze()
#temp1

In [332]:
# or apply this cell
tmpk = mpcalc.smooth_n_point(temp['air_temp'].squeeze())
uwnd = mpcalc.smooth_n_point(uv_wind['u'].squeeze())
vwnd = mpcalc.smooth_n_point(uv_wind['v'].squeeze())
#tmpk

In [333]:
# Get time in a nice datetime object format
vtime = temp.time.values.astype('datetime64[ms]').astype('O')[0]
#vtime

In [334]:
# Combine 1D latitude and longitudes into a 2D grid of locations
lon_2d, lat_2d = np.meshgrid(lon, lat)
# Gridshift for barbs
lon_2d[lon_2d > 180] = lon_2d[lon_2d > 180] - 360

In [335]:
lats = ds.lat.data
lons = ds.lon.data

In [341]:
# Calculate variable dx, dy values for use in calculations
dx, dy = mpcalc.lat_lon_grid_deltas(lon_var, lat_var)
#dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

C:\Users\Samuel-NUIST\anaconda3\lib\site-packages\metpy\xarray.py:1305: UserWarning: Argument longitude given as xarray Variable...casting to its data. xarray DataArrays are recommended instead.
  warnings.warn(
C:\Users\Samuel-NUIST\anaconda3\lib\site-packages\metpy\xarray.py:1305: UserWarning: Argument latitude given as xarray Variable...casting to its data. xarray DataArrays are recommended instead.
  warnings.warn(


In [342]:
tadv = mpcalc.advection(temp1, u_wind, v_wind,(dx, dy))

ValueError: Must provide dx, dy, and/or dz arguments or input DataArray with interpretable dimension coordinates.

In [ ]:
# Calculate temperature advection using metpy function
temp_adv2 = mpcalc.advection(temp1 * units.degC, [u_wind, v_wind],
                       (dx, dy), dim_order='yx') * units('degC/sec')

In [ ]:
# Calculate temperature advection using metpy function
temp_adv1 = mpcalc.advection(tmpk, (uwnd, vwnd),
                            (dx, dy), dim_order='yx').to_base_units()